<img src="https://github.com/slt666666/FAO_lecture/blob/main/title.png?raw=true" alt="title" height="300px">


# QTL-seq - practice -

We will perform QTL-seq analysis using simulation data & perform sliding window analysis using yesterday's result in this practice part.

It may help you to understand ...
* the process of QTL-seq analysis 
* How to interpret the results of QTL-seq analysis
* What data is required for QTL-seq
* What is sliding window analysis & Why it is required.

## The contents in this notebook ... 

* Review of QTL-seq

* Practice - QTL-seq analysis for simulation data -

  * We assume very simple organism & conduct QTL-seq analysis to understand the process of QTL-seq.

* Introduction of QTL-Seq pipeline (Github)

* Sliding window analysis
  * What is sliding window analysis for MutMap & QTL-seq?
  * We use published data in MutMap paper [(Abe et al., 2012)](https://www.nature.com/articles/nbt.2095) & perform sliding window analysis.

# Main contents

# Review of QTL-seq

  QTL-seq analysis is one the methods to identify the genomic region which is associate with the quantitative traits. (like QTL mapping.)

* MutMap: Qualitative trait

* QTL-seq: Quantitative trait

The brief process of QTL-seq is below:

1. Cross 2 cultivars that showed different phenotype to generate F2 population and survey phenotypes of F2  progenies.

1. If there are segregtion, Select 20~ samples each showing opposite phenotypes (high and low).

1. Collect bulked DNA from selected samples for each phenotype (Low bulk & High bulk), and perform next generation sequencing (bulk sequencing).

1. Sequence reads are aligned to the reference genome of one parent. And then, calculate SNP-index for whole genome region based on alignment results.

1. Finally, to compare SNP-index between high bulk and low bulk, calculate ΔSNP-index which indicate the difference of SNP-index between high and low bulks. As a results, the genomic region which allele frequency is hugely different between high and low bulk are identified as high ΔSNP-index(0 <) or low ΔSNP-index(< 0).

This genomic region is candidate of causative region that affect phenotype.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/QTLseq.png?raw=true" alt="title" height="750px">

In this practice lecture, we plan to experience each step using simulation data to understand the process of QTL-seq.

# Practice: QTLseq analysis using very simple simulation data

!! please run the below code, this code downloads programs. !!

In [ ]:
# Prepare modules & packages
!wget -O module_qtlseq.py https://github.com/slt666666/FAO_lecture/blob/main/module_qtlseq.py?raw=true

from module_qtlseq import make_2_cultivars
from module_qtlseq import make_F2_progeny
from module_qtlseq import check_distribution
from module_qtlseq import high_and_low_bulk_sequencing
from module_qtlseq import alignment
from module_qtlseq import calculate_SNP_index
from module_qtlseq import visualize_SNP_index
from module_qtlseq import calculate_delta_SNP_index
from module_qtlseq import visualize_delta_SNP_index
from module_qtlseq import check_results
from module_qtlseq import qtl_seq_simulation
from module_qtlseq import get_yesterday_SNP_index
from module_qtlseq import visualize_SNP_index2
from module_qtlseq import sliding_window

## Simulation (default setting):

* We assume the plant which has only 1 chromosome & 100 bp.

* There are 2 cultivars, cultivar A & cultivar B. The height of cultivar B is shorter than cultivar A.

* Cultivar B has 40 SNPs that are different from reference genotype.

* One of 40 SNPs has a big effect on the plant height. Other SNPs also have small effect.

(It's not realistic, just simulation.)

<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation5.png?raw=true" alt="title" height="200px">

Please tun the below code to make 2 cultivars !!


In [ ]:
cultivar_A, cultivar_B = make_2_cultivars(length=100, snp=40)
print("cultivar A (tall):", cultivar_A)
print("cultivar B (short):", cultivar_B)

## 1. Cross parent lines and generate F2 progeny

To identify a causative SNP that affect plant height of cultivar A & B, we perform QTL-seq analysis.

At first, we make F2 population by crossing cultivar A and cultivar B to perform QTL-seq.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation6.png?raw=true" alt="title" height="300px">

The phenotypes of F2 progenies are randomly distributed depending on their genotypes.

<br>

Please run the below code to generate F2 progenies between Cultivar A and B.

The below code generates 200 F2 progenies.

In [ ]:
f2_progeny = make_F2_progeny(cultivar_A, cultivar_B, progeny=200)
check_distribution(f2_progeny)

※ Until here(Generate F2 progeny), the process are experimental part in reality.

```
※ The above program generates genotype of progeny which is randomly mixed between cultivar A and cultivar B.

Then, program decide the phenotype values based on the simulated effect of each SNP genotype (& This program consider the error variance).

So, the program saved the genotype data of simulated population & the effect of each SNP.

Therefore, we can check the result of QTL-seq is correct or not by comparing simulated data.
```

## 3. Bulk sequence of samples with high phenotype & low phenotype.

If there is a SNP that only high(/low) phenotype samples have, this SNP is causative.

So, to identify which SNP has significant effect, we select sample sets which showed high phenotype and low phenotype.

Then, we perform bulked sequencing for each sample set.

In this notebook, we collect top 20 samples and bottom 20 samples for phenotype values.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation7.png?raw=true" alt="title" height="400px">


Please run the below code to perform bulk sequencing for each sample set!!

Below code generate sequence read file for each bulk.

In [ ]:
high_reads, low_reads = high_and_low_bulk_sequencing(f2_progeny, top=20, bottom=20, reads=500)

You can check these fastq files using file system ↓

```
How to check files in your Google Colab server space.
1. Click the file icon in upper left.
2. The file list showed files in your server space.
(3. if there is no "high_bulked_sequences.fastq" and "low_bulked_sequences.fastq", please click the third icon from the right
```
<img src="https://github.com/slt666666/FAO_lecture/blob/main/filesystem.png?raw=true" alt="title" height="250px">

## 4. Calculate SNP index

To check are there SNPs that only high(/low) phenotype samples have,  we calculate SNP-index for each bulk.

We will align these sequence reads to the **cultivar_A** genotype and calculate SNP-index.

Based on SNP-index of high-bulk and low-bulk, we identify the SNP that only one of bulked samples have.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation8.png?raw=true" alt="title" height="300px">


### Alignment reads to the reference sequence

To calculate SNP-index, we need to align sequences to the cultivar A genome.

Then, we calculate the number of variants in bulked DNA for each SNP position.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation9.png?raw=true" alt="title" height="400px">

Please run the below code to align bulk sequence reads to cultivar A genotype !!


In [ ]:
high_bulk_alignment_result, low_bulk_alignment_result = alignment(high_reads, low_reads, cultivar_A)


```
※ In this notebook, we perform alignment of sequence reads by our program.

But basically, when we perform alignment, we use mapping tool such as BWA(http://bio-bwa.sourceforge.net/).

So, if you conduct QTL-seq analysis by your own data, it may has required to use mapping tool.

But QTL-seq pipeline that we will introduce later contains mapping tools. So, you don't need to care about it.
```

### Calculate SNP index based on alignment results for low bulk & high bulk

After the alignment, we can calculate SNP index for each bulk.

In this study, SNP index showed the ratio of the allele which is different from cultivar_A for each SNP position.

If SNP index is close to 1, the ratio is biased (almost all samples has a SNP that is different from cultivar_A genotype).

<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation12.png?raw=true" alt="title" height="250px">

Please run the below code to calculate SNP-index for each bulk !!

In [ ]:
high_bulk_SNP_index, low_bulk_SNP_index = calculate_SNP_index(high_bulk_alignment_result, low_bulk_alignment_result, cultivar_A, cultivar_B)
print("High bulk")
display(high_bulk_SNP_index)
print("Low bulk")
display(low_bulk_SNP_index)

## 5. Visulalize SNP index plot for each bulk samples

After calculating SNP index, visualize it to see the SNP position that allele frequency is biased to the each bulk.


In [ ]:
visualize_SNP_index(high_bulk_SNP_index, low_bulk_SNP_index)

## 6. Calculate ΔSNP-index

These above graphs showed there are several high SNP-index positions in each bulk.

But positions that showed high SNP index in both high and low bulk may not be candidates.

So, we should remove this kind of positions and identify the high SNP-index position that only High or Low bulk samples has.

To do that, we calculate ΔSNP-index.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation11.png?raw=true" alt="title" height="600px">

Please run the below code to caculate ΔSNP-index !!

In [ ]:
delta_SNP_index = calculate_delta_SNP_index(high_bulk_SNP_index, low_bulk_SNP_index)
delta_SNP_index

## 7. Visulalize ΔSNP-index plot

After calculating ΔSNP-index, visualize it to search the causative position.

In [ ]:
visualize_delta_SNP_index(delta_SNP_index)

Red circle showed the position of ΔSNP-index > 0.9.

## check analysis results & real genotype

QTL-seq analysis showed causative SNP that has ΔSNP index is almost 1.

The below code showed the effect of each SNP & calculated ΔSNP-index (sorted by SNP effect.)

Try to check QTL-seq result is correct or not !!

In [ ]:
check_results(delta_SNP_index)

The setting of this simulation was ... 1 SNP has big effect on phenotype (+10).

QTLseq analysis success to identify the position using only reference fasta & high and low bulked sequencing data!!

# Play with simulation!

You can specify
- the length of reference genome
- the number of mutations
- the number of progeny
- the number of reads

please make different situation & perform MutMap analysis using below code.

In [ ]:
qtl_seq_simulation(length=100, snp=40, progeny=200, reads=500)

# Introduction of QTL-seq pipeline

Our laboratory developped the very simple pipeline to conduct QTL-seq.

<img src="https://github.com/YuSugihara/QTL-seq/blob/master/images/1_logo.png?raw=true" alt="title" height="100px">

(https://github.com/YuSugihara/QTL-seq)

This pipeline is very simple to use.

To use this pipeline, the required input data is ...

The required input data is ...
* reference fasta
* reference sequence reads
* High bulk DNA sequence reads 
* Low bulk DNA sequence reads

And you just install pipeline & use below command. That's it !

```
# command
qtlseq -r reference.fasta -p reference.fastq -b1 high_bulked_sequences.fastq -b2 low_bulked_sequences.fastq -n1 20 -n2 20 -o output_name
```

<br>

```
ex) The public avaliable reference genome of Rice is Nipponbare cultivar.

But if you want to use different cultivar like Hitomebore cultivar as a reference, reference.fastq is required.
```

# Sliding window analysis

In this part, we use yesterday's MutMap result in 2nd practice to experience sliding window analysis.


## Review of Sliding Window analysis

Different from simulation study, the real data (experimental data) does not always showed theoretically correct values.

The real data usually contains errors, outliers, human error...etc.

For MutMap(QTLseq) analysis, SNP-index will be different from expected values due to the errors.

ex) When the read depth is not enough,

sometimes it could lead a big outlier like SNP-index=0(or SNP-index=1) in the not important SNP position.

### Sliding window

To escape these errors, one approach is taking the average of SNP-indexes of nearby regions.

It will diminish the influence of error and result in a more correct value.

Applying sliding window analysis to MutMap/QTL-seq, we can calculate mean values of SNP-index in the specific interval and identify regions that show high/low average SNP-index.

As a result, we can get highly reliable SNP-index results.


<img src="https://github.com/slt666666/FAO_lecture/blob/main/simulation13.png?raw=true" alt="title" height="400px">

The process of sliding window analysis is...

1. Decide the size of interval (window size), and movement size to next interval (step size)
1. Extract data (SNP-indexes) in the interval.
1. Calculate average value of them.
1. Move the interval by the step size.
1. Continue 2-4 process to finish the whole region.

## Dataset

We used yesterday's results of SNP-index.

**Materials & Methods are same as yesterday.**


### Materials

We used rice cultivar "Hitomebore" as a reference. 

"Hit1917-pl1" cultivar was generated by mutagenesis and showed lightgreen leaf color (low chlorophyll content).

Hit1917-pl1 has almost 1500 SNPs that are different from reference.


### Methods

To identify which SNP variant is causative for the change of leaf color, we conducted MutMap analysis.

We generated over 200 F2 progenies by crossing reference and mutant lines.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/material.png?raw=true" alt="title" height="400px">

### Bulked sequencing & alignment

Then, we conducted bulked sequencing of progenies that showed mutant phenotype.

(You can access sequence data in [DRA000499](https://www.ncbi.nlm.nih.gov/sra?term=DRA000499) as a reference.)

After that, we aligned these sequences and calculate SNP index.

In this notebook, we used [table data](https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_chr10.txt) that showed alignment results (the SNP information) in the chromosome10.

<img src="https://github.com/slt666666/FAO_lecture/blob/main/sample_data.png?raw=true" alt="title" height="300px">



Please run the below code, to download the yesterday result (alignment & SNP calling results & SNP-index) !!



In [ ]:
!wget -q -O mutmap_dataset.txt https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_chr10.txt
SNP_index = get_yesterday_SNP_index()
SNP_index

## Visualize SNP index plot

Below code visualize original SNP index plot. (same plot as yesterday.)

- x axis: position in the chromosome（column:`POS`）
- y axis: SNP index（column:`SNP_index`）


In [ ]:
visualize_SNP_index2(SNP_index)

Red circles showed high SNP-index (>0.8). 

There are multiple positions showed high SNP-index.

To extract more reliable regions from this results, we try to perform sliding window analysis.

## Sliding window analysis & visualize it.

Below code perform sliding window analysis.

Window size & Step size to perform sliding window analysis can be set.

```
At first, we set window size = 1mbp & step size = 200kbp
(considering that the legth of chromosome10 is 23207287bp.)
```

In [ ]:
sliding_window(SNP_index, window_size=1000000, step_size = 200000)

Red line showed average SNP-index for each interval.

Plot showed 22~23Mbp in chromosome 10 showed high average SNP-index.

This result suggests that there is a gene that related to leaf color in 22~23Mbp, chromosome 10.

In this region, there is a gene "OsCAO1" that codes Chlorophyllide a oxygenase.

And it was known that the knock-out line of OsCAO1 showed light green leaf. (Abe et al., 2012)。


## Our developped MutMap/QTL-seq analysis pipeline include sliding window analysis

You can specify window size & step size in these pipelines.

- MutMap
https://github.com/YuSugihara/MutMap#Usage

- QTL-seq
https://github.com/YuSugihara/QTL-seq#usage

---
## Summary

In this notebook, we demonstrate **QTL-seq** analysis using simulation data.

And **Sliding window** analysis using published data.

Actually, MutMap & QTL-seq analysi is very simple & easy methods.

<br>

The important point is how to interpret the results (SNP-index values).

We should take care of false positives, errors, ...etc by applying sliding window analysis, checking read depth of interested SNPs...etc

<br>

Tomorrow, we'll plan to demonstrate **genomic prediction**.